# 📊 Analyse des Stratégies DCA Optimisées

Ce notebook présente les résultats de l'optimisation Bayésienne des stratégies DCA.

## Objectifs
1. Visualiser les courbes d'équité des meilleures stratégies
2. Comparer **Rebalancing vs Accumulation Pure**
3. Analyser l'importance des hyperparamètres
4. Valider les résultats Walk-Forward

In [ ]:
# Imports
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import optuna
from pathlib import Path

# Modules locaux
from data_loader import load_data, compute_indicators, prepare_simulation_data
from strategy_core import simulate_dca_strategy, compare_strategies, DCAStrategyParams
from metrics import compute_all_metrics, format_metrics_table, max_drawdown, calmar_ratio

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
RESULTS_DIR = Path('../results')

## 1️⃣ Chargement des Données

In [ ]:
# Configuration
TICKER = "SPY"  # Modifier selon vos besoins
START_DATE = "2015-01-01"
END_DATE = "2024-01-01"
MONTHLY_AMOUNT = 500.0

# Charger les données
print(f"Chargement de {TICKER}...")
prices, indicators, is_bull, dates = prepare_simulation_data(TICKER, START_DATE, END_DATE)

print(f"\n📊 Données chargées:")
print(f"   - Période: {dates[0].strftime('%Y-%m-%d')} → {dates[-1].strftime('%Y-%m-%d')}")
print(f"   - {len(prices)} jours de trading")
print(f"   - {is_bull.sum() / len(is_bull) * 100:.1f}% en Bull Market")

## 2️⃣ Comparaison: Rebalancing vs Accumulation

Comparons les deux approches fondamentales:
- **Accumulation Pure**: Ne jamais vendre, accumuler indéfiniment
- **Rebalancing**: Prendre des profits quand RSI > seuil

In [ ]:
# Exécuter les deux stratégies
results_accum, results_rebal = compare_strategies(
    prices, indicators, is_bull, 
    monthly_amount=MONTHLY_AMOUNT
)

# Calculer les métriques
metrics_accum = compute_all_metrics(
    results_accum['equity_curve'], 
    results_accum['total_invested']
)
metrics_rebal = compute_all_metrics(
    results_rebal['equity_curve'], 
    results_rebal['total_invested']
)

# Afficher le tableau comparatif
comparison_df = pd.DataFrame({
    'Métrique': [
        'Total Investi (€)',
        'Valeur Finale (€)',
        'Profit (€)',
        'Profit (%)',
        'CAGR (%)',
        'Max Drawdown (%)',
        'Calmar Ratio',
        'Sortino Ratio',
        'Nb Achats',
        'Nb Ventes'
    ],
    'Accumulation': [
        f"{metrics_accum['total_invested']:,.0f}",
        f"{metrics_accum['final_value']:,.0f}",
        f"{metrics_accum['profit']:,.0f}",
        f"{metrics_accum['profit_pct']:.1f}%",
        f"{metrics_accum['cagr']:.1f}%",
        f"{metrics_accum['max_drawdown']:.1f}%",
        f"{metrics_accum['calmar_ratio']:.2f}",
        f"{metrics_accum['sortino_ratio']:.2f}",
        results_accum['n_buys'],
        results_accum['n_sells']
    ],
    'Rebalancing': [
        f"{metrics_rebal['total_invested']:,.0f}",
        f"{metrics_rebal['final_value']:,.0f}",
        f"{metrics_rebal['profit']:,.0f}",
        f"{metrics_rebal['profit_pct']:.1f}%",
        f"{metrics_rebal['cagr']:.1f}%",
        f"{metrics_rebal['max_drawdown']:.1f}%",
        f"{metrics_rebal['calmar_ratio']:.2f}",
        f"{metrics_rebal['sortino_ratio']:.2f}",
        results_rebal['n_buys'],
        results_rebal['n_sells']
    ]
})

# Déterminer le gagnant pour chaque métrique
comparison_df['Gagnant'] = ['—'] * len(comparison_df)
comparison_df.loc[4, 'Gagnant'] = '🏆 Accum' if metrics_accum['cagr'] > metrics_rebal['cagr'] else '🏆 Rebal'
comparison_df.loc[5, 'Gagnant'] = '🏆 Accum' if metrics_accum['max_drawdown'] < metrics_rebal['max_drawdown'] else '🏆 Rebal'
comparison_df.loc[6, 'Gagnant'] = '🏆 Accum' if metrics_accum['calmar_ratio'] > metrics_rebal['calmar_ratio'] else '🏆 Rebal'

print("\n" + "="*70)
print("📊 COMPARAISON: ACCUMULATION vs REBALANCING")
print("="*70)
display(comparison_df)

In [ ]:
# Graphique des courbes d'équité
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Courbes d\'Équité', 'Drawdown'),
    vertical_spacing=0.12,
    row_heights=[0.7, 0.3]
)

# Courbes d'équité
fig.add_trace(
    go.Scatter(
        x=dates, y=results_accum['equity_curve'],
        name='Accumulation',
        line=dict(color='#2E86AB', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=dates, y=results_rebal['equity_curve'],
        name='Rebalancing',
        line=dict(color='#E94F37', width=2)
    ),
    row=1, col=1
)

# Total investi (ligne de référence)
invested_curve = np.zeros(len(prices))
invested_curve[0] = MONTHLY_AMOUNT
for i in range(1, len(prices)):
    if i % 21 == 0:  # Nouveau mois
        invested_curve[i] = invested_curve[i-1] + MONTHLY_AMOUNT
    else:
        invested_curve[i] = invested_curve[i-1]

fig.add_trace(
    go.Scatter(
        x=dates, y=invested_curve,
        name='Total Investi',
        line=dict(color='gray', width=1, dash='dash')
    ),
    row=1, col=1
)

# Drawdown
from metrics import drawdown_series

dd_accum = drawdown_series(results_accum['equity_curve']) * 100
dd_rebal = drawdown_series(results_rebal['equity_curve']) * 100

fig.add_trace(
    go.Scatter(
        x=dates, y=-dd_accum,
        name='DD Accumulation',
        fill='tozeroy',
        line=dict(color='#2E86AB', width=1)
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=dates, y=-dd_rebal,
        name='DD Rebalancing',
        fill='tozeroy',
        line=dict(color='#E94F37', width=1)
    ),
    row=2, col=1
)

fig.update_layout(
    title=f'Comparaison des Stratégies DCA - {TICKER}',
    height=700,
    template='plotly_dark',
    showlegend=True,
    legend=dict(x=0.01, y=0.99)
)

fig.update_yaxes(title_text='Valeur (€)', row=1, col=1)
fig.update_yaxes(title_text='Drawdown (%)', row=2, col=1)

fig.show()

## 3️⃣ Résultats de l'Optimisation Optuna

Chargeons les résultats de l'optimisation (si disponibles).

In [ ]:
# Charger l'étude Optuna si elle existe
study_path = RESULTS_DIR / 'optuna_study.db'

if study_path.exists():
    # Lister les études disponibles
    study_summaries = optuna.study.get_all_study_summaries(
        storage=f"sqlite:///{study_path}"
    )
    
    print("📚 Études disponibles:")
    for summary in study_summaries:
        print(f"   - {summary.study_name}: {summary.n_trials} trials")
    
    # Charger la dernière étude
    if study_summaries:
        latest_study = study_summaries[-1].study_name
        study = optuna.load_study(
            study_name=latest_study,
            storage=f"sqlite:///{study_path}"
        )
        print(f"\n✅ Étude chargée: {latest_study}")
        print(f"   Meilleur score: {study.best_value:.4f}")
else:
    print("⚠️ Aucune étude Optuna trouvée.")
    print("   Exécutez d'abord: python src/optuna_search.py")
    study = None

In [ ]:
# Visualiser l'importance des hyperparamètres
if study is not None and len(study.trials) > 10:
    fig = optuna.visualization.plot_param_importances(study)
    fig.update_layout(
        title='Importance des Hyperparamètres',
        template='plotly_dark'
    )
    fig.show()

In [ ]:
# Visualiser l'historique d'optimisation
if study is not None:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.update_layout(
        title='Historique d\'Optimisation',
        template='plotly_dark'
    )
    fig.show()

In [ ]:
# Top 10 des meilleures configurations
if study is not None:
    trials_df = study.trials_dataframe()
    trials_df = trials_df.sort_values('value', ascending=False)
    
    print("\n🏆 TOP 10 CONFIGURATIONS")
    print("="*80)
    
    top_10 = trials_df.head(10)[[
        'number', 'value', 
        'params_enable_rebalancing',
        'params_dca_multiplier',
        'params_signal_threshold',
        'user_attrs_final_value'
    ]].copy()
    
    top_10.columns = ['Trial', 'Score', 'Rebalancing', 'Multiplier', 'Threshold', 'Final Value']
    display(top_10)

## 4️⃣ Analyse de Sensibilité

In [ ]:
# Sensibilité au multiplicateur DCA
multipliers = range(1, 13)
results_by_mult = []

for mult in multipliers:
    params = DCAStrategyParams(
        monthly_amount=MONTHLY_AMOUNT,
        enable_rebalancing=False,
        dca_multiplier=mult,
        signal_indicator=0,  # RSI
        signal_threshold=30
    )
    
    results = simulate_dca_strategy(
        prices, indicators, is_bull,
        params.monthly_amount, params.initial_war_chest,
        params.enable_rebalancing, params.rebalance_rsi_trigger,
        params.rebalance_profit_trigger, params.rebalance_pct,
        params.use_regime_filter, params.bear_multiplier_reduction,
        params.dca_multiplier, params.signal_indicator,
        params.signal_threshold, params.cooldown_months
    )
    
    equity = results[0]
    years = len(equity) / 252.0
    
    results_by_mult.append({
        'multiplier': mult,
        'final_value': equity[-1],
        'calmar': calmar_ratio(equity, years),
        'max_dd': max_drawdown(equity) * 100
    })

sens_df = pd.DataFrame(results_by_mult)

# Graphique
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Valeur Finale vs Multiplicateur', 'Calmar Ratio vs Multiplicateur')
)

fig.add_trace(
    go.Bar(x=sens_df['multiplier'], y=sens_df['final_value'], name='Valeur Finale'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=sens_df['multiplier'], y=sens_df['calmar'], mode='lines+markers', name='Calmar'),
    row=1, col=2
)

fig.update_layout(
    title='Analyse de Sensibilité: Multiplicateur DCA',
    template='plotly_dark',
    height=400
)
fig.show()

print(sens_df.to_string(index=False))

## 5️⃣ Comparaison Multi-Indices

In [ ]:
# Comparer sur plusieurs indices
TICKERS = ['SPY', 'QQQ']  # Ajouter d'autres si souhaité

multi_results = []

for ticker in TICKERS:
    try:
        prices_t, indicators_t, is_bull_t, dates_t = prepare_simulation_data(
            ticker, START_DATE, END_DATE
        )
        
        results_a, results_r = compare_strategies(
            prices_t, indicators_t, is_bull_t, MONTHLY_AMOUNT
        )
        
        multi_results.append({
            'Ticker': ticker,
            'Accum Final': results_a['equity_curve'][-1],
            'Rebal Final': results_r['equity_curve'][-1],
            'Gagnant': 'Accum' if results_a['equity_curve'][-1] > results_r['equity_curve'][-1] else 'Rebal'
        })
    except Exception as e:
        print(f"Erreur pour {ticker}: {e}")

if multi_results:
    multi_df = pd.DataFrame(multi_results)
    print("\n📊 COMPARAISON MULTI-INDICES")
    display(multi_df)

## 6️⃣ Conclusion & Recommandations

In [ ]:
# Résumé final
print("\n" + "="*70)
print("📋 RÉSUMÉ & RECOMMANDATIONS")
print("="*70)

print(f"\n📈 Meilleure stratégie sur {TICKER}:")

if metrics_accum['calmar_ratio'] > metrics_rebal['calmar_ratio']:
    print("   🏆 ACCUMULATION PURE")
    print(f"   → Calmar Ratio: {metrics_accum['calmar_ratio']:.2f}")
    print(f"   → CAGR: {metrics_accum['cagr']:.1f}%")
    print(f"   → Le marché haussier favorise l'accumulation sans vente")
else:
    print("   🏆 REBALANCING")
    print(f"   → Calmar Ratio: {metrics_rebal['calmar_ratio']:.2f}")
    print(f"   → CAGR: {metrics_rebal['cagr']:.1f}%")
    print(f"   → La prise de profits améliore le ratio risque/rendement")

print("\n💡 Paramètres recommandés:")
print("   - Multiplieur DCA: 5-8x lors des corrections")
print("   - Seuil RSI achat: 25-35")
print("   - Cooldown: 2-3 mois après achat levier")
print("   - Filtre de régime: Activé (réduire levier en Bear Market)")